<a href="https://colab.research.google.com/github/tazeenida/Ethical-AI/blob/feature_collabe_hw2/Case1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pickle

import pandas as pd

from sklearn.linear_model import LogisticRegression

from sklearn import preprocessing



with open('/content/BEST_LR_MODEL_EVER.pkl', 'rb') as f:

  clf = pickle.load(f)

df = pd.read_csv('/content/test_bias_new.csv')

X = df.loc[:, df.columns != 'approved']

Y = df['approved']

norm_X = preprocessing.normalize(X, axis=0)


print("New Test Score:", clf.score(norm_X, Y))


New Test Score: 0.475
